## Initialization

In [43]:
import numpy as np

In [44]:
magnObj = 60
nrCamPix = 16 # num pixels behind lenslet
camPixPitch = 6.5
microLensPitch = nrCamPix*camPixPitch/magnObj
voxPitch = microLensPitch/5

The number of voxels along each side length of the cube is determined by the voxPitch. An odd number of voxels will allow the center of a voxel in the center of object space.
Object space center:
voxCtr:center voxel where all rays of the central microlense converge
volCtr:same center in micrometers

In [45]:
voxNrX = round(250/voxPitch)
if voxNrX % 2 == 1:
    voxNrX += 1
voxNrYZ = round(700/voxPitch)
if voxNrYZ % 2 == 1:
    voxNrYZ += 1
voxCtr = np.array([voxNrX/2, voxNrYZ/2, voxNrYZ/2])
volCtr = voxCtr*voxPitch
volCtr

array([125.14666667, 350.13333333, 350.13333333])

In [46]:
wavelength = 0.550
naObj = 1.2
nMedium = 1.52


Finding angles to/between central lenset. Basically, the angle going to each of the 16 pixels for each microlens.

In [47]:
microLensCtr = [8, 8] # (unit: camera pixels)
rNA = 7.5 # radius of edge of microlens lens (unit:camera pixels), 
            # can be measured in back focal plane of microlenses
camPixRays = np.zeros([nrCamPix, nrCamPix])
i = np.linspace(1, nrCamPix, nrCamPix)
j = np.linspace(1, nrCamPix, nrCamPix)
jv, iv = np.meshgrid(i, j) # row/column defined instead of by coordinate
distFromCtr = np.sqrt((iv-0.5-microLensCtr[0])**2 + (jv-0.5-microLensCtr[1])**2)
camPixRays[distFromCtr > rNA] = np.NaN
iRel2Ctr = iv-0.5-microLensCtr[0]
jRel2Ctr = jv-0.5-microLensCtr[1]
camPixRaysAzim = np.round(np.rad2deg(np.arctan2(jRel2Ctr, iRel2Ctr)))
camPixRaysAzim[distFromCtr > rNA] = np.NaN
distFromCtr[distFromCtr > rNA] = np.NaN
camPixRaysTilt = np.round(np.rad2deg(np.arcsin(distFromCtr/rNA*naObj/nMedium)))

# posRel2Ctr[distFromCtr > rNA] = np.NaN
# camPixRaysAzim = np.arctan2(jv-0.5-microLensCtr[1], iv-0.5-microLensCtr[0])/(np.pi/180)
# for i in range(nrCamPix):
#     for j in range(nrCamPix):
#         if distFromCtr[i,j] > rNA:
#             camPixRays[i,j] = np.NaN
#         else:
#             pass

Camera ray entrance. For each inital ray position, we find the position on the entrance face of the object cube for which the ray enters.
This is bascially the same as "rayEnter". Here $x=0$.

In [48]:
camRayEntranceX = np.zeros([nrCamPix, nrCamPix])
camRayEntranceY = volCtr[0]*np.tan(np.deg2rad(camPixRaysTilt))*np.sin(np.deg2rad(camPixRaysAzim))+volCtr[1]
camRayEntranceZ = volCtr[0]*np.tan(np.deg2rad(camPixRaysTilt))*np.cos(np.deg2rad(camPixRaysAzim))+volCtr[2]
camRayEntranceX[np.isnan(camRayEntranceY)] = np.NaN
camRayEntranceY[1,6]
# When accessing array, x & y orientation are the same, 
# but minus one needs to be applied to match up with the mathematica indices.

nrRays = np.sum(~np.isnan(camRayEntranceY)) # Number of all rays in use

In [59]:
camRayEntrance = np.array([camRayEntranceX, camRayEntranceY, camRayEntranceZ])
camRayEntrance[:,1,6]
rayEnter = camRayEntrance.copy()
volCtrGridTemp = np.array([np.full((nrCamPix,nrCamPix), volCtr[i]) for i in range(3)])
rayExit = rayEnter + 2 * (volCtrGridTemp - rayEnter)
rayEnter[:,4,6]

array([  0.        , 328.36223018, 298.84382846])

Testing the rayExit generate to only one element. This is the first element shown in the Mathematica code.

In [50]:
# rayExit = np.zeros((3,16,16))
print(rayEnter[:,1,4])
# rayExit[:,1,4] = rayEnter[:,1,4] + 2*(volCtr-rayEnter[:,1,4])
print(rayExit[:,1,4])

[  0.         277.57966622 213.67973144]
[250.29333333 422.68700045 486.58693523]


Direction of the rays at the exit plane

In [51]:
rayDiff = rayExit - rayEnter
# Efficient normalizing of rayDiff vectors
rayDiff = rayExit - rayEnter
mags = np.linalg.norm(rayDiff, axis=0)
rayDiff = rayDiff / mags
# # Ineffcient normalizing
# # Normalize the rayDiff vectors (can be done w/o for loops)
# for i in range(nrCamPix):
#     for j in range(nrCamPix):
#         rayDiff[:,i,j] = rayDiff[:,i,j] / np.linalg.norm(rayDiff[:,i,j])
print(rayDiff[:,1,4])

[0.62932039 0.36484793 0.68617916]


In [52]:
def rotation_matrix(axis, angle):
    '''Generates the rotation matrix that will rotate a 3D vector
    around "axis" by "angle" counterclockwise.'''
    ax, ay, az = axis[0], axis[1], axis[2]
    s = np.sin(angle)
    c = np.cos(angle)
    u = 1 - c
    R_tuple = ( ( ax*ax*u + c,    ax*ay*u - az*s, ax*az*u + ay*s ),
        ( ay*ax*u + az*s, ay*ay*u + c,    ay*az*u - ax*s ),
        ( az*ax*u - ay*s, az*ay*u + ax*s, az*az*u + c    ) )
    R = np.asarray(R_tuple)
    return R
    

In [53]:
# Note: might only need two perp rays, not matrix or ray again, would call it rayDir
def calc_rayUnitVectors(ray):
    '''
    Allows to the calculations to be done in ray-space coordinates
    as oppossed to laboratory coordinates
    Parameters:
        ray (np.array): normalized 3D vector giving the direction 
                        of the light ray
    Returns:
        ray (np.array): same as input
        ray_perp1 (np.array): normalized 3D vector
        ray_perp2 (np.array): normalized 3D vector
        R (np.array): 3x3 rotation matrix form ray to lab frame
    '''
    theta = np.arccos(np.dot(ray, np.array([1,0,0])))
    # Unit vectors that give the laboratory axes
    scope_axis = np.array([1,0,0])
    scope_perp1 = np.array([0,1,0])
    scope_perp2 = np.array([0,0,1])
    theta = np.arccos(np.dot(ray, scope_axis))
    print(f"Rotating by {np.around(np.rad2deg(theta), decimals=0)} degrees")
    normal_vec = np.cross(ray, scope_axis) / np.linalg.norm(np.cross(ray, scope_axis))
    R = rotation_matrix(normal_vec, theta)
    Rinv = rotation_matrix(normal_vec, -theta)
    # Extracting basis vectors that are orthogonal to the ray and will be parallel
    # to the laboratory axes that are not the optic axis after a rotation.
    # Note: If scope_perp1 if the y-axis, then ray_perp1 if the 2nd column of Rinv.
    ray_perp1 = np.dot(Rinv, scope_perp1)
    ray_perp2 = np.dot(Rinv, scope_perp2)

    return [ray, ray_perp1, ray_perp2, R]

Testing rayUnitVectors function

In [54]:
from random import Random
rng = Random()
a = (rng.uniform(-5,5), rng.uniform(-5,5), rng.uniform(-5,5))
# ray = np.array([0.5,0.5,0])
ray = a / np.linalg.norm(a)
calc_rayUnitVectors(ray)
calc_rayUnitVectors(rayDiff[:,1,4])

Rotating by 53.0 degrees
Rotating by 51.0 degrees


[array([0.62932039, 0.36484793, 0.68617916]),
 array([-0.36484793,  0.9183009 , -0.15365366]),
 array([-0.68617916, -0.15365366,  0.71101949]),
 array([[ 0.62932039,  0.36484793,  0.68617916],
        [-0.36484793,  0.9183009 , -0.15365366],
        [-0.68617916, -0.15365366,  0.71101949]])]

In [55]:
# print(yv)
# print(camPix)
# print(distFromCtr)
print(camPixRaysTilt)
print(camPixRaysAzim)
print(type(camPixRaysAzim[0,0]))
print(camPixRays)

[[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan 51. 47. 45. 43. 43. 45. 47. 51. nan nan nan nan]
 [nan nan nan 48. 43. 39. 37. 36. 36. 37. 39. 43. 48. nan nan nan]
 [nan nan 48. 42. 37. 33. 30. 28. 28. 30. 33. 37. 42. 48. nan nan]
 [nan 51. 43. 37. 31. 27. 24. 22. 22. 24. 27. 31. 37. 43. 51. nan]
 [nan 47. 39. 33. 27. 22. 18. 16. 16. 18. 22. 27. 33. 39. 47. nan]
 [nan 45. 37. 30. 24. 18. 13. 10. 10. 13. 18. 24. 30. 37. 45. nan]
 [nan 43. 36. 28. 22. 16. 10.  4.  4. 10. 16. 22. 28. 36. 43. nan]
 [nan 43. 36. 28. 22. 16. 10.  4.  4. 10. 16. 22. 28. 36. 43. nan]
 [nan 45. 37. 30. 24. 18. 13. 10. 10. 13. 18. 24. 30. 37. 45. nan]
 [nan 47. 39. 33. 27. 22. 18. 16. 16. 18. 22. 27. 33. 39. 47. nan]
 [nan 51. 43. 37. 31. 27. 24. 22. 22. 24. 27. 31. 37. 43. 51. nan]
 [nan nan 48. 42. 37. 33. 30. 28. 28. 30. 33. 37. 42. 48. nan nan]
 [nan nan nan 48. 43. 39. 37. 36. 36. 37. 39. 43. 48. nan nan nan]
 [nan nan nan nan 51. 47. 45. 43. 43. 45. 47. 51. nan nan nan 

In [56]:
# Set ray and voxel parameters
a = (rng.uniform(-5,5), rng.uniform(-5,5), rng.uniform(-5,5))
opticAxis = a / np.linalg.norm(a)
opticAxis = [1,0,0]
a = (rng.uniform(-5,5), rng.uniform(-5,5), rng.uniform(-5,5))
ray = a / np.linalg.norm(a)
# ray = [0,1,0]
Delta_n = rng.uniform(-1,1)
Delta_n = 1

Forming the product of the Jones matrices

In [57]:
def voxRayJM(Delta_n, opticAxis, rayDir, ell):
    '''Compute Jones matrix associated with a particular ray and voxel combination'''
    azim = np.arctan2(np.dot(opticAxis, rayDir[1]), np.dot(opticAxis, rayDir[2]))
    # add dependence of azim on birefringence
    print(f"Azimuth angle of index ellipsoid is {np.around(np.rad2deg(azim), decimals=0)} degrees.")
    ret = abs(Delta_n) * (1 - np.dot(opticAxis, rayDir[0]) ** 2) * 2 * np.pi * ell/ wavelength
    print(f"Accumulated retardance from index ellipsoid is {np.around(np.rad2deg(ret), decimals=0)} ~ {int(np.rad2deg(ret)) % 360} degrees.")
    offdiag = 1j * np.sin(2 * azim) * np.sin(ret / 2)
    diag1 = np.cos(ret / 2) + 1j * np.cos(2 * azim) * np.sin(ret / 2)
    diag2 = np.conj(diag1)
    return np.matrix([[diag1, offdiag], [offdiag, diag2]])

def rayJM(JMlist):
    '''Computes product of Jones matrix sequence
    Equivalent method: np.linalg.multi_dot([JM1, JM2])
    '''
    product = np.identity(2)
    for JM in JMlist:
        product = product @ JM
    return product

Example usage of voxRayJM and rayJM

In [58]:
rayUnitVectors = calc_rayUnitVectors(ray)
rayDir = rayUnitVectors[0:3]
ell = 1
JM = voxRayJM(Delta_n, opticAxis, rayDir, ell)
print(f"Jones matrix is \n{np.around(JM, decimals=2)}")

JM1 = voxRayJM(Delta_n, opticAxis, rayDir, ell)
JM2 = voxRayJM(Delta_n, opticAxis, rayDir, ell/2)
rayJM([JM1, JM2])

Rotating by 128.0 degrees
Azimuth angle of index ellipsoid is 43.0 degrees.
Accumulated retardance from index ellipsoid is 405.0 ~ 44 degrees.
Jones matrix is 
[[-0.92-0.03j -0.  -0.38j]
 [-0.  -0.38j -0.92+0.03j]]
Azimuth angle of index ellipsoid is 43.0 degrees.
Accumulated retardance from index ellipsoid is 405.0 ~ 44 degrees.
Azimuth angle of index ellipsoid is 43.0 degrees.
Accumulated retardance from index ellipsoid is 202.0 ~ 202 degrees.


matrix([[ 5.52089737e-01-0.06960082j,  3.49568195e-18-0.83087463j],
        [-2.65640629e-18-0.83087463j,  5.52089737e-01+0.06960082j]])

In [81]:
import import_ipynb
from ray_siddon import *

For a single microlens

In [ ]:
start = rayEnter[:,i,j]
stop = rayExit[:,i,j]
siddon_list = siddon_params(start, stop, [voxPitch]*3, [voxNrX, voxNrYZ, voxNrYZ])
seg_mids = siddon_midpoints(start, stop, siddon_list)
voxels_of_segs = voxel_indices(seg_mids, voxPitch)
ell_in_voxels = siddon_lengths(start, stop, siddon_list)

In [ ]:
ray = rayDiff[:,i,j]
rayUnitVectors = calc_rayUnitVectors(ray)
rayDir = rayUnitVectors[0:3]
JM_list = []
for m in len(ell_in_voxels):
    ell = ell_in_voxels[m]
    vox = voxels_of_segs[m]
    Delta_n, opticAxis = get_ellipsoid(vox)
    JM = voxRayJM(Delta_n, opticAxis, rayDir, ell)
    JM_list.append(JM)
effective_JM = rayJM(JM_list)